In [2]:
import tensorflow as tf
import numpy as np
import pydub

In [ ]:
#Dataset preprocessing
def load_wav_16k_mono(filename):
    a = pydub.AudioSegment.from_mp3(filename)
    y = np.array(a.get_array_of_samples())
    return y


def preprocess(file_path):
    wav = load_wav_16k_mono("data/train/" + file_path)
    wav = wav.astype(np.float32, order='C')
    spectrogram = tf.signal.stft(wav, frame_length=160, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    #print(spectrogram.shape)
    spectrogram = np.pad(np.asarray(spectrogram), ((0,2500-spectrogram.shape[0]), (0,0), (0, 0)), 'constant')
    return spectrogram



X = []
Y = []

n = 0
n_0 = 0
with open("data/train_gt.csv", 'r') as file:
    for line in file:
        audio_file, label = line.split(',')
        if (int(label) == 0 and n_0 <= 3234) or int(label) == 1:
            X.append(preprocess(audio_file))
            Y.append(int(label))
            if (int(label) == 0):
                n_0 += 1
            print(n)
            n += 1


In [ ]:
X

In [ ]:
a = 0
b = 0
for i in Y:
    if i == 1:
        a += 1
    else:
        b += 1
print(a, b)

In [25]:
del X[1]
del Y[1]

In [7]:
Y = np.array(Y)
X = np.stack(X, axis=0)

In [28]:
np.save('Train.npy', X)
np.save('TrainY.npy', Y)

In [2]:
X = np.load('Train.npy')
Y = np.load('TrainY.npy')

In [ ]:
X.shape

In [ ]:
Y.shape

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((X, Y))

In [ ]:
dataset